In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
tf.__version__

'2.10.1'

# Preprocess the Traning dataset

- **`rescale=1./255`**:
  - This parameter rescales the pixel values of the images from a range of 0-255 to a range of 0-1. Dividing the pixel values by 255 normalizes them, making it easier for the neural network to process and learn from the images.
  
- **`shear_range=0.2`**:
  - This parameter applies a random shear transformation to the images. A shear transformation tilts the image at a specific angle (in radians), which helps the model generalize better to different perspectives of the image.
  
- **`zoom_range=0.2`**:
  - This parameter zooms in or out on the images by a factor of up to 20%. It helps the model learn features from various scales, enhancing its robustness.

- **`horizontal_flip=True`**:
   - This parameter flips the images horizontally with a 50% chance. It helps the model learn features irrespective of orientation, improving its generalization ability when objects appear from different angles.



**`target_size=(64, 64)`**:
- **Purpose**:
  - Resizes all images to a fixed size of **64x64 pixels**.
  - Ensures that all images have consistent dimensions for input to the model.



**`batch_size=32`**:
- **Purpose**:
  - Specifies the number of images per batch.
  - Images are processed in groups of 32 during training, which helps manage memory efficiently.



**`class_mode='binary'`**:
- **Purpose**:
  - Defines the type of classification problem:
    - `'binary'`: For binary classification (e.g., two classes like cats vs. dogs).
    - `'categorical'`: For multi-class classification.
  - Here, the labels will be binary (0 or 1).

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

training_set = train_datagen.flow_from_directory(
    'CNN/dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)



Found 8000 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_set = test_datagen.flow_from_directory(
    "CNN/dataset/test_set",
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


# Building CNN

In [ ]:
cnn = tf.keras.models.Sequential()

## convolution layer

`filters=32`:

- Defines the number of filters (also called kernels) used in the convolutional layer.
- Each filter is responsible for learning to detect different features (e.g., edges, textures) in the input images.
- In this case, 32 filters are applied, meaning the output of this layer will have 32 feature maps.

`kernel_size=3`:

- Specifies the size of the convolutional filter.
- A 3x3 kernel is commonly used in image processing because it balances performance and computational cost.
- The kernel slides across the image to extract various features like edges, shapes, and textures.

`activation='relu'`:

- Applies the ReLU (Rectified Linear Unit) activation function to introduce non-linearity into the model.
- ReLU sets all negative values in the feature maps to zero and keeps positive values unchanged.
- This helps the network learn more complex patterns and reduces the likelihood of vanishing gradients.

`input_shape=[64, 64, 3]`:

- Specifies the shape of the input image.
- The input shape is 64x64x3, where:
 - 64x64 is the height and width of the image in pixels.
 - 3 represents the number of color channels (RGB) in the image.
- This parameter is required only for the first layer in the model.

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

## Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

## second convolution layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

## flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

## Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

## OUTPUT layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

# Training the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.fit(training_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 [==============================] - 160s 405ms/step - loss: 0.6618 - accuracy: 0.6018 - val_loss: 0.6027 - val_accuracy: 0.6990
Epoch 2/25
250/250 [==============================] - 14s 56ms/step - loss: 0.5854 - accuracy: 0.6956 - val_loss: 0.5462 - val_accuracy: 0.7360
Epoch 3/25
250/250 [==============================] - 13s 51ms/step - loss: 0.5477 - accuracy: 0.7269 - val_loss: 0.5152 - val_accuracy: 0.7390
Epoch 4/25
250/250 [==============================] - 13s 52ms/step - loss: 0.5160 - accuracy: 0.7421 - val_loss: 0.5030 - val_accuracy: 0.7575
Epoch 5/25
250/250 [==============================] - 13s 51ms/step - loss: 0.4890 - accuracy: 0.7601 - val_loss: 0.4861 - val_accuracy: 0.7675
Epoch 6/25
250/250 [==============================] - 13s 51ms/step - loss: 0.4742 - accuracy: 0.7717 - val_loss: 0.4936 - val_accuracy: 0.7660
Epoch 7/25
250/250 [==============================] - 13s 54ms/step - loss: 0.4623 - accuracy: 0.7725 - val_loss: 0.4602 - val_accurac

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img("CNN\dataset\single_prediction\cat_or_dog_2.jpg", target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = np.around(cnn.predict(test_image / 255))
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 14ms/step


`test_image = image.load_img("CNN\dataset\single_prediction\cat_or_dog_2.jpg", target_size=(64, 64)` :
- load image and convert pixel size to 64 x 64

`test_image = image.img_to_array(test_image)` :
- convert image into numpy array

`test_image = np.expand_dims(test_image, axis=0)` :
- add extra dimention to the array

`result = np.around(cnn.predict(test_image / 255))`:
- `test_image / 255` : Normalizes pixel values to the range [0.1]
- `cnn.predict()` : make prediction
- `np.around(cnn.predict(test_image / 255))` : round prediction to the nearest integer (0,1)

In [ ]:
prediction

'cat'